In [61]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [62]:
# IMPORTS

import datasets
import platform
import psutil
import json
import os
import random
import torch
import transformers
import warnings

from datetime import datetime
from importlib import metadata
from IPython.display import HTML
from jinja2 import Template

In [63]:
# GET COMPUTER SPECS

platform_info = {}

platform_info['physical_cpu_cores'] = psutil.cpu_count(logical=False)
platform_info['total_cpu_cores'] = psutil.cpu_count(logical=True)

def get_available_device():
    """Helper method to find best possible hardware to run
    Returns:
        torch.device used to run experiments.
        str representation of backend.
    """
    # Check if CUDA is available
    if torch.cuda.is_available():
        return torch.device("cuda"), "cuda"

    # Check if ROCm is available
    if torch.version.hip is not None and torch.backends.mps.is_available():
        return torch.device("rocm"), "rocm"

    # Check if MPS (Apple Silicon) is available
    if torch.backends.mps.is_available():
        return torch.device('cpu'), "mps"

    # Fall back to CPU
    return torch.device("cpu"), "cpu"

# Check device info
device, backend = get_available_device()

# Check for GPU-specific details if CUDA or ROCm is available
if device.type == "cuda":
    cuda_device_count = torch.cuda.device_count()
    cuda_device_name = torch.cuda.get_device_name(0)
    cuda_version = torch.version.cuda
elif device.type == "rocm":
    cuda_device_count = torch.cuda.device_count()
    cuda_device_name = torch.cuda.get_device_name(0)
    cuda_version = torch.version.hip
else:
    cuda_device_count = 0
    cuda_device_name = "N/A"
    cuda_version = "N/A"

platform_info['device'] = device.type
platform_info['backend'] = backend
platform_info['cuda_device_count'] = cuda_device_count
platform_info['cuda_device_name'] = cuda_device_name
platform_info['cuda_version'] = cuda_version

print(json.dumps(platform_info, indent=4))


{
    "physical_cpu_cores": 4,
    "total_cpu_cores": 8,
    "device": "cpu",
    "backend": "cpu",
    "cuda_device_count": 0,
    "cuda_device_name": "N/A",
    "cuda_version": "N/A"
}


In [64]:
# PROMPT TEMPLATES

prompt_templates = {}

template1 = Template("""\
Create a square album cover for the following album:

Album name : {{ album.name }}
Artist{% if album.artists|length > 1 %}s{% endif %} : {{ album.artists | join(', ') }}
Release Date : {{ album.date }}
Label : {{ album.label }}
{% if album.genres %}Genres : {{ album.genres | join(', ') }}{% endif %}
Tracks:
{% for track in album.tracks %}- {{ track.name }}\n{% endfor %}
""")
prompt_templates['1-long'] = template1

In [65]:
# GET ALBUM DATA

file_id = "" # if need for a specific album, put the file name here

if file_id == "":
    album_files = os.listdir('input/albums')
    random_album_file = random.choice(album_files)
else:
    random_album_file = f'{file_id}.json'

with open(f'input/albums/{random_album_file}', 'r') as file:
    album_data = json.load(file)

In [66]:
# GET SPECIFIC PROMPT

for prompt_template in prompt_templates.values():
    prompt = prompt_template.render(album=album_data)

In [ ]:
run_info = {}
run_info['computer_specs'] = platform_info
run_info['album_id'] = album_data['id']

for key, template in prompt_templates.items():
    curr_run_info = run_info.copy()
    run_info['prompt_key'] = key

    prompt = template.render(album=album_data)

    print(json.dumps(curr_run_info, indent=4))

    del curr_run_info

{
    "computer_specs": {
        "physical_cpu_cores": 4,
        "total_cpu_cores": 8,
        "device": "cpu",
        "backend": "cpu",
        "cuda_device_count": 0,
        "cuda_device_name": "N/A",
        "cuda_version": "N/A"
    },
    "album_id": "1qeIKM4E2eLToXvOzW2civ",
    "prompt_key": "1-long"
}
